In [1]:
# =======
# imports
# =======
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import numpy as np
import os

In this step we want to calculate the maximum deployment of offgrid technologies. We do so in QGIS by intersecting the access layers with the technology layers, and counting the number of people in the intersection. Then we calculate the max deployment by scaling the total off-grid potential (i.e. offgrid demand) by the ratio between the number of people in the intersection and the total population in the offgrid area: 

$$
Max_{tech} = offPOP_{intersection} \times \frac{ offDEM }{ offPOP_{tot} }
$$

Diesel and PV are actually available everywhere (at different costs). So the only tech for which a constraint makes sense is hydro.

In the raster calculator we use this formula for the various target years:

(( "access_2010@1" \* "hy@1" ) = 0 ) \* 0 + (( "access_2010@1" \* "hy@1" ) !=  0 ) \* "access_pop_2010@1"

The result is a serie of rasters with the number of people that can access offgrid hydro (note, it's always 2010 people, it needs to be normalized later!!!).
We then run zonal statistics to count the people in each polygon. The result is saved in wtp_vec.shp. Let's import it here:

In [2]:
gis_dir = "C:\\Users\\dallalongaf\\OneDrive - TNO\\work\\2016 02 - TransRisk\\4 - energy access\\AFR_alt_2\\gis_layers\\constraints\\"
wtp = gpd.read_file(gis_dir+"wtp_vec.shp")
wtp.head()

,ID,CODE,COUNTRY,CNTRY,REGION,countrycou,countrysum,countrymea,Country Na,wtp-2010,...,pop2010c_4,pop2010s_4,pop_count,pop_sum,hy2010_sum,hy2020_sum,hy2030_sum,hy2040_sum,hy2050_sum,geometry
0,1,ALG,Algeria,Algeria,North Africa,2325210.0,407605757.0,175.298471,Algeria,142,...,2325210.0,25050021.0,2325210.0,25050021.0,964.0,964.0,964.0,964.0,964.0,"POLYGON ((1894589.42156076 6093196.823427028, ..."
1,2,ANG,Angola,Angola,Central Africa,1247523.0,248776781.0,199.416589,Angola,142,...,1247523.0,15306412.0,1247523.0,15307500.0,95.0,825599.0,825599.0,825599.0,825617.0,"POLYGON ((3809405.5247603 2283690.127493567, 3..."
2,3,ANG,Angola,Angola,Central Africa,7152.0,1699358.0,237.605984,Angola,142,...,7152.0,214475.0,7152.0,214484.0,28.0,1620.0,1620.0,1620.0,1620.0,"POLYGON ((3744504.513407562 2674551.841173306,..."
3,4,ANG,Angola,Angola,Central Africa,102.0,21784.0,213.568627,Angola,142,...,102.0,0.0,102.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((3650051.721446254 1343900.192709396,..."
4,5,ANG,Angola,Angola,Central Africa,21.0,4608.0,219.428571,Angola,142,...,21.0,57.0,21.0,57.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((3755607.506678306 2543061.0734161, 3..."


The population with access to hydro is in columns hy[YEAR]_sum.

In [3]:
# create a summary per country
wtpc = wtp.groupby('COUNTRY').sum().reset_index()
wtpc.head()

,COUNTRY,countrycou,countrysum,countrymea,wtp-2010,wtp-2020,wtp-2030,wtp-2040,wtp-2050,wtp-2060,...,pop2010s_3,pop2010c_4,pop2010s_4,pop_count,pop_sum,hy2010_sum,hy2020_sum,hy2030_sum,hy2040_sum,hy2050_sum
0,Algeria,2.325210e+06,4.076058e+08,175.298471,142,168,204,243,283,323,...,2.505002e+07,2.325210e+06,2.505002e+07,2.325210e+06,2.505002e+07,964.0,964.0,964.0,964.0,964.0
1,Angola,1.254848e+06,2.503181e+08,-207849.857601,2556,3186,4194,5382,6588,7794,...,1.552171e+07,1.254848e+06,1.552186e+07,1.254848e+06,1.552295e+07,123.0,827219.0,827219.0,827219.0,827237.0
2,Benin,1.163010e+05,2.350732e+07,202.124814,132,175,225,278,337,406,...,8.054508e+06,1.163010e+05,8.054520e+06,1.163010e+05,8.054520e+06,0.0,594807.0,594807.0,595910.0,595916.0
3,Botswana,5.806660e+05,1.152753e+08,198.522476,81,98,117,144,170,198,...,1.333993e+06,5.806660e+05,1.336533e+06,5.806660e+05,1.336563e+06,0.0,0.0,0.0,19188.0,19264.0
4,Burkina Faso,2.750980e+05,4.958767e+07,180.254549,23,31,40,49,60,72,...,2.920000e+02,2.750980e+05,6.730000e+02,2.750980e+05,1.290668e+07,0.0,154.0,154.0,154.0,181.0


In [4]:
# Per country normalize population with hydro access by total population (hy[YEAR] / pop_sum)
hy_norm = pd.DataFrame()
hy_norm['COUNTRY'] = wtpc['COUNTRY']

hy_col = [c for c in wtpc.columns if 'hy' in c]
off_pop_col = [c for c in wtpc.columns if 'pop2010' in c and 's' in c]

for i in range(len(hy_col)):
    y = hy_col[i][2:6]
    hy_norm.loc[:,y] = wtpc.loc[:,hy_col[i]] / wtpc.loc[:,off_pop_col[i]]

hy_norm = hy_norm.fillna(0)
hy_norm.head()

,COUNTRY,2010,2020,2030,2040,2050
0,Algeria,0.000039,0.000039,0.000039,0.000038,0.000038
1,Angola,0.000013,0.057870,0.053294,0.053294,0.053295
2,Benin,0.000000,0.458896,0.073946,0.073985,0.073985
3,Botswana,0.000000,0.000000,0.000000,0.014384,0.014413
4,Burkina Faso,0.000000,0.527397,0.527397,0.527397,0.268945


In [5]:
# check that they are all < 1
(hy_norm>1).sum()

COUNTRY    53
2010        0
2020        0
2030        0
2040        0
2050        0
dtype: int64

Values in hy_norm are the number of people with access to off-grid hydro, normalized by the total number of people with off-grid access. this is what we need to plug into the formula at the top of the document.  

Let's first import the off-grid demand to make the final calculation.

In [6]:
off_dem = pd.read_excel("off-grid_demand.xlsx")
off_dem.head()

,COUNTRY,2005,2010,2020,2030,2040,2050
0,Algeria,0,0,21.760154,30.080734,39.594472,49.627058
1,Angola,0,0,38.585148,78.016721,132.621450,208.283081
2,Benin,0,0,0.379780,3.349794,5.203164,7.708840
3,Botswana,0,0,0.000000,0.000000,0.000000,0.000000
4,Burkina Faso,0,0,0.000114,0.000193,0.000580,0.001345


In [7]:
max_hy_PJ = off_dem.set_index('COUNTRY') * hy_norm.set_index('COUNTRY')
max_hy_PJ = max_hy_PJ.fillna(0)
max_hy_PJ.head()

,2005,2010,2020,2030,2040,2050
COUNTRY,,,,,,
Algeria,0.0,0.0,0.000841,0.001160,0.001524,0.001910
Angola,0.0,0.0,2.232929,4.157848,7.067970,11.100442
Benin,0.0,0.0,0.174279,0.247702,0.384954,0.570341
Botswana,0.0,0.0,0.000000,0.000000,0.000000,0.000000
Burkina Faso,0.0,0.0,0.000060,0.000102,0.000306,0.000362


In [8]:
# import region names
data_dir = "C:\\Users\\dallalongaf\\OneDrive - TNO\\work\\2016 02 - TransRisk\\4 - energy access\\AFR_alt\\code\\input_data\\"
regions = pd.read_excel(data_dir+'country_mapping.xlsx',sheet_name='regions')
regions.head()

,REG-TIAM_15,REG-TIAM_36 Region Name,COUNTRY_ISO,Country Name,COUNTRY_UNPD,COUNTRY,REG-TIAM_all,Color code,Group,Unnamed: 9,Country dict,Unnamed: 11,Region dict,Unnamed: 13,Unnamed: 14,"""COUNTRY_ISO"",""REG-TIAM_all"""
0,AFR,Western Africa,XXX,(blank),(blank),(blank),AWE,#B3CD1C,Africa,"'XXX': ['XXX','(blank)', '(blank)'],","'XXX': ['XXX','(BLANK)', '(BLANK)'],",NaN,"'XXX': 'AWE',",NaN,NaN,"""XXX"",""AWE"""
1,AFR,Algeria,DZA,Algeria,Algeria,Algeria,DZA,#F6EF80,Africa,"'DZA': ['DZA','Algeria', 'Algeria'],","'DZA': ['DZA','ALGERIA', 'ALGERIA'],",NaN,"'DZA': 'DZA',",NaN,NaN,"""DZA"",""DZA"""
2,AFR,Angola,AGO,Angola,Angola,Angola,AGO,#B693C4,Africa,"'AGO': ['AGO','Angola', 'Angola'],","'AGO': ['AGO','ANGOLA', 'ANGOLA'],",NaN,"'AGO': 'AGO',",NaN,NaN,"""AGO"",""AGO"""
3,AFR,Western Africa,BEN,Benin,Benin,Benin,AWE,#B3CD1C,Africa,"'BEN': ['BEN','Benin', 'Benin'],","'BEN': ['BEN','BENIN', 'BENIN'],",NaN,"'BEN': 'AWE',",NaN,NaN,"""BEN"",""AWE"""
4,AFR,Southern Africa,BWA,Botswana,Botswana,Botswana,ASO,#EA4F58,Africa,"'BWA': ['BWA','Botswana', 'Botswana'],","'BWA': ['BWA','BOTSWANA', 'BOTSWANA'],",NaN,"'BWA': 'ASO',",NaN,NaN,"""BWA"",""ASO"""


In [9]:
# group by tiam region 
def group_by_reg(df):
    """
    Group data by summing over TIAM regions.
    Returns grouped DF.
    """
    ret = df.reset_index()\
            .merge(regions[['COUNTRY','REG-TIAM_all']],on='COUNTRY')\
            .groupby('REG-TIAM_all').sum()#.transpose()
    return ret
max_hy_PJ_reg = group_by_reg(max_hy_PJ)
max_hy_PJ_reg.head()

,2005,2010,2020,2030,2040,2050
REG-TIAM_all,,,,,,
ACE,0.0,0.0,0.434414,3.383587,5.611931,8.356116
AEA,0.0,0.0,1.212377,2.184589,3.479225,6.340199
AGO,0.0,0.0,2.232929,4.157848,7.067970,11.100442
ASE,0.0,0.0,0.000000,0.000000,8.812029,14.659322
ASO,0.0,0.0,0.000000,0.000000,0.000081,0.017704


In [10]:
max_hy_PJ_reg.to_excel("hydro_cnstr.xlsx")